# QAACCOUNT 账户类

QA_Account() 是quantaxis的核心类, 其作用是一个可以使用规则兼容各种市场的账户类


## 1. 调用方式

In [2]:
import QUANTAXIS as QA
QA.QA_Account()

< QA_Account Acc_E5bFQXlT>

## 2.参数详解

In [6]:
# strategy_name=None,
# user_cookie=None,
# portfolio_cookie=None,
# account_cookie=None,
# market_type=MARKET_TYPE.STOCK_CN,
# frequence=FREQUENCE.DAY,
# broker=BROKER_TYPE.BACKETEST,
# init_hold={},
# init_cash=1000000,
# commission_coeff=0.00025,
# tax_coeff=0.001,
# margin_level={},
# allow_t0=False,
# allow_sellopen=False,
# allow_margin=False,
# running_environment=RUNNING_ENVIRONMENT.BACKETEST

## 3. 基于规则实例化

基于不同市场的不同规则, 我们可以实例化不同的账户类

- 允许保证金交易:  allow_marigin = True

- 允许买入后当日卖出: allow_t0 = True

- 允许卖空开仓(裸卖空): allow_sellopen= True

### 3.1 股票普通账户

In [8]:
stock_account= QA.QA_Account(allow_t0=False,allow_margin=False,allow_sellopen=False,running_environment=QA.MARKET_TYPE.STOCK_CN)

### 3.2 股票融资融券账户

In [11]:
rzrq_account = QA.QA_Account(allow_t0=False,allow_margin=True,allow_sellopen=True,running_environment=QA.MARKET_TYPE.STOCK_CN)

### 3.3 期货账户

In [14]:
future_account = QA.QA_Account(allow_t0=True,allow_margin=True,allow_sellopen=True, running_environment=QA.MARKET_TYPE.FUTURE_CN)

### 3.4 期权账户

In [16]:
option_account = QA.QA_Account(allow_t0=True,allow_margin=True,allow_sellopen=True, running_environment=QA.MARKET_TYPE.OPTION_CN)

## 3.5 其他市场账户


In [17]:
xxx = QA.QA_Account(allow_t0=True,allow_margin=True,allow_sellopen=True, running_environment=QA.MARKET_TYPE.CRYPTOCURRENCY)

## 4. 账户的初始资金/初始仓位

默认账户是无仓位, 默认现金 1000000 RMB


In [19]:
stock_account.init_assets

{'cash': 1000000, 'hold': {}}

In [20]:
stock_account.init_cash

1000000

In [21]:
stock_account.init_hold

Series([], Name: amount, dtype: float64)

### 4.1 在实例化的时候初始化仓位信息

使用json/dict的格式初始化  

```python
# init_hold参数
init_hold={code1:amount1,code2:amount2}
```

实例化完 会显示在 account.hold中

[注意] 在t+1的账户中, 初始化仓位依然可以当日交易

In [32]:
stock_account= QA.QA_Account(init_hold={'000001':500}, allow_t0=False,allow_margin=False,allow_sellopen=False,running_environment=QA.MARKET_TYPE.STOCK_CN)

In [33]:
stock_account.init_assets

{'cash': 1000000, 'hold': {'000001': 500}}

In [34]:
stock_account.hold

code
000001    500
Name: amount, dtype: int64

In [36]:
stock_account.sell_available

code
000001    500
Name: amount, dtype: int64

## 4.2 在实例化的时候初始现金信息

```python
# init_cash 参数
init_cash= 200000
```


In [38]:
stock_account= QA.QA_Account(init_cash= 200000,init_hold={'000001':500}, allow_t0=False,allow_margin=False,allow_sellopen=False,running_environment=QA.MARKET_TYPE.STOCK_CN)

In [39]:
stock_account.init_assets

{'cash': 200000, 'hold': {'000001': 500}}

In [40]:
stock_account.init_cash

200000

In [41]:
stock_account.cash

[200000]

In [42]:
stock_account.cash_available

200000

### 4.3 在已经实例化的账户中修改现金/ 重置现金操作

此操作无法撤销

- 现金记录全部消除
- 账户的持仓不会消除



In [45]:
stock_account.reset_assets(init_cash=50000)

In [46]:
stock_account.init_assets

{'cash': 50000, 'hold': {'000001': 500}}

In [47]:
stock_account.cash

[50000]

In [48]:
stock_account.cash_available

50000